# Train a BERT classifier

In [ ]:
import pickle
from bert_sklearn import BertClassifier
from bert_sklearn import load_model
import pandas as pd
import pathlib
from tools import processing

#### Load data

In [ ]:
corpus = "stories/" # Options: "combined/" or "stories/"
dataset_df = pd.read_pickle("../data/" + corpus + "train.pkl")

#### Train BERT classifiers

In [ ]:
dFolders = {"targetExpression": "../models/classifiers/" + corpus + "targetExpression/",
            "context3wmasked": "../models/classifiers/" + corpus + "context3wmasked/",
            "context3w": "../models/classifiers/" + corpus + "context3w/"}

for col in dFolders:
    
    model = BertClassifier()
    
    pathlib.Path(dFolders[col]).mkdir(parents=True, exist_ok=True)

    X = dataset_df[col].tolist()
    y = dataset_df["animated"].tolist()

    model.fit(X, y)

    # save model to disk
    savefile= dFolders[col] + "/bert.bin"
    model.save(savefile)
    print("DONE")
    print()